In [93]:
from scanDir import scanDir
import numpy as np
import cv2
import os
import blend_modes



In [199]:
input_dir=r'E:\5Apr18.LaiRegnault.SEP92.RS.biceps_3pec_delt'
video_dict = {}
video_dict['A'], video_dict['B'] = scanDir(input_dir, "avi")
codec = 0
#substitute 0 in call to videowriter for uncompressed, otherwise cv2.VideoWriter_fourcc(*'mp4v')
##https://www.microsoft.com/en-us/p/hevc-video-extensions-from-device-manufacturer/9n4wgh0z6vhq?activetab=pivot:overviewtab


Found file with extension .avi: E:\5Apr18.LaiRegnault.SEP92.RS.biceps_3pec_delt\Run1retpro1_95-3-4msCam1.avi
Found file with extension .avi: E:\5Apr18.LaiRegnault.SEP92.RS.biceps_3pec_delt\Run1retpro1_95-3-4msCam2.avi


In [193]:
video_dict

{'A': 'E:\\5Apr18.LaiRegnault.SEP92.RS.biceps_3pec_delt\\Run1retpro1_95-3-4msCam1.avi',
 'B': 'E:\\5Apr18.LaiRegnault.SEP92.RS.biceps_3pec_delt\\Run1retpro1_95-3-4msCam2.avi'}

In [22]:
os.path.splitext(os.path.basename(video_dict['A']))[0]

'Run1retpro1_95-3-4msCam1'

In [200]:
capA = cv2.VideoCapture(video_dict['A'])
capB = cv2.VideoCapture(video_dict['B'])
input_name =os.path.splitext(os.path.basename(video_dict['A']))
output_name = "RGBMerge_"+input_name[0][:-4]+input_name[1]
out = cv2.VideoWriter(output_name,
                     codec,
                     round(capA.get(5),2),(int(capA.get(3)), int(capA.get(4))))
while(capA.isOpened()):
    retA, frameA = capA.read()
    retB, frameB = capB.read()
    if retA == True:
        ## give frames an alpha channel to prepare for blending; blend_modes requires 32bit
        frameA = cv2.cvtColor(frameA, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
        frameB = cv2.cvtColor(frameB, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
        AdiffB = blend_modes.difference(frameA,frameB,1)
        ## get rid of alpha channel in preparation for converting back to grayscale; opencv prefers 8bit
        frameA = cv2.cvtColor(frameA, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
        frameB = cv2.cvtColor(frameB, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
        AdiffB = cv2.cvtColor(AdiffB, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
        ## convert to grayscale so we can merge into 3-channe
        frameA = cv2.cvtColor(frameA, cv2.COLOR_BGR2GRAY)  
        frameB = cv2.cvtColor(frameB, cv2.COLOR_BGR2GRAY)  
        AdiffB = cv2.cvtColor(AdiffB, cv2.COLOR_BGR2GRAY)  
                  
        merged = cv2.merge((AdiffB, frameB, frameA))
        cv2.imshow('merged',merged)
        out.write(merged)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
capA.release()
capB.release()
out.release()
cv2.destroyAllWindows()
print("done!")

                


done!


In [165]:
testC = r"C:\Users\Phil\Development\possumpolish\RGBMerge_Run1retpro1_95-3-4ms.avi"
capC = cv2.VideoCapture(testC)

while(capC.isOpened()):
    retC, frameC = capC.read()
    if retC == True:
        B, G, R = cv2.split(frameC)
        cv2.imshow('R', R)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
capC.release()

cv2.destroyAllWindows()
print("done!")

done!


In [155]:
print(r'C:\Users\Phil\Development\possumpolish'+output_name)

C:\Users\Phil\Development\possumpolishRGBMerge_Run1retpro1_95-3-4ms.avi


In [ ]:
def concatenateVideos(video_files, codec, interval=1):
    frame_index = 0
    video_index = 0
    cap = cv2.VideoCapture(video_files[0])
    fourcc = cv2.VideoWriter_fourcc(*codec)
    output_name = "concatenated_1in"+str(interval)+".avi"
    out = cv2.VideoWriter(output_name, 
                          cv2.VideoWriter_fourcc(*vid_codec), 
                          cap.get(5), (1280,720))
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame_index += 1
        if frame is None:
            print("end of video " + str(video_index) + " ... next one now")
            video_index += 1
            if video_index >= len(video_files):
                break
            cap = cv2.VideoCapture(video_files[ video_index ])
            ret, frame = cap.read()
            frame_index = 0
        elif frame_index == interval:
            frame = frame.astype(np.uint8)
            cv2.imshow('frame',frame)
            out.write(frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            frame_index = 0         
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("done!")
